# Importación de librerias

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-fznqj6wg
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-req-build-fznqj6wg
  Created wheel for tensorflow-examples: filename=tensorflow_examples-6ae97eaf3dbd607ed3eccf18f7dc05d7a3b677e3_-py3-none-any.whl size=299717 sha256=4d6584f7610dcdb3c92dce26263298c45351944236979c47a0050c56a12d1ac0
  Stored in directory: /tmp/pip-ephem-wheel-cache-dn41psay/wheels/4e/f5/c2/bfe75b834c9028b2529023bf74541c46ead531b513a8010d21
Failed to build tensorflow-examples
    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/i

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import plotly.graph_objects as px

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Creación de variables y fuciones

## Ajustes visualización

In [ ]:
def one_hot_mask(y):
    ''' Do the one hot encoding for the masks.
  
    Arguments:
        - y (tf tensor): Mask of shape (height, width, 3)

    Returns:
        - mask (tf tensor): Mask after do the one hot. Shape (height, width, num_classes) '''

    one_hot_map = []
    for color in colors:
        class_map = tf.reduce_all(tf.equal(y, color), axis = -1)
        one_hot_map.append(class_map)
    mask = tf.cast(tf.stack(one_hot_map, axis = -1), tf.int32)
    return mask


def load_image(folder, file, height = 96, width = 256, crop = False):
    ''' Load and preprocess a train image by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Normalize the image to [0, 1]
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed image '''

    # Load the image (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.float32)

    # Crop the image
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the image
    image = tf.image.resize(image, (height, width))

    # Normalize the image
    image = tf.cast(image, tf.float32)/255.0
    return image


def load_mask(folder, file, height = 96, width = 256, one_hot = True, crop = False):
    ''' Load and preprocess a train mask by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Reshaping the mask from (height, width, 3) to (height, width, 30): One hot encoding
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - one_hot (bool): Do one hot encoding or not -- True
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed mask '''

    # Load the mask (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.int32)

    # Crop the mask
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the mask
    image = tf.image.resize(image, (height, width), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # One hot encoding
    if one_hot:
        image = one_hot_mask(image)
    return image


def one_hot_to_color_mask(mask, colors, img_height = 96, img_width = 256):
    ''' Convert from the mask from the classes with highest probablities to the correct color. From (96, 256, 1) to (96, 256, 3).
  
    Arguments:
        - mask (tf tensor): Mask with the classes with highest probabilities
        - colors (list): List with the class colors
        - img_height (int): Height of the images -- 96
        - img_width (int): Width of the images -- 256

    Returns:
        - color_mask (tf tensor): Color mask '''

    color_mask = np.zeros((img_height, img_width, channels)).astype('float')
    for c in range(len(colors)):
        color_true = mask == c
        for i in range(3):
            color_mask[:,:,i] += color_true*colors[c][i]

    color_mask = tf.cast(color_mask, dtype = tf.int32)

    return color_mask

def load_train(image_name, mask_name):
    ''' Load and preprocess a train image and its mask
  
    Arguments:
        - image_name (string): Name of the image to load
        - mask_name (string): Name of the mask to load

    Returns:
        - image (tf tensor): Preprocessed image
        - mask (tf tensor): Preprocessed mask '''

    image = load_image(train_images_folder_path, image_name, img_height, img_width)
    mask = load_mask(train_mask_folder_path, mask_name, img_height, img_width)
    return image, mask

In [ ]:
train_images_folder_path = "/content/drive/My Drive" 
train_mask_folder_path = "/content/drive/MyDrive/GTA Segmentacion" 

img_height, img_width, channels = 96, 256, 3

batch_size = 2

colors = np.array([(0, 0, 0), (111, 74, 0), (81, 0, 81), (128, 64, 128), (244, 35, 232), (250, 170, 160), (230, 150, 140), (70, 70, 70), (102, 102, 156), (190, 153, 153), (180, 165, 180), 
                   (150, 100, 100), (150, 120, 90), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), ( 0, 0, 142), 
                   ( 0, 0, 70), (0, 60, 100), (0, 0, 90), (0, 0, 110), (0, 80, 100), (0, 0, 230), (119, 11, 32), (0, 0, 142)], dtype = np.int32)

model = tf.keras.models.load_model('/content/drive/My Drive/Modelo_segmentacion_UNET/best_model_weights_and_architecture')

# Inicialización

In [ ]:
path_result = "/content/drive/MyDrive/Resultados/"
path_GTA = "/content/drive/MyDrive/GTA/"

rango = ["Original", "Best NST", "CycleGAN-6", "CycleGAN-10", "CycleGAN-12"]
imgs = ["01526.png", "00031.png", "00513.png", "01166.png", "01151.png", "01782.png", "01550.png", "01450.png", "00557.png", "02078.png",  # 10 primeros
        "01575.png", "00370.png", "01401.png", "01999.png", "00203.png", "00560.png", "00142.png", "02074.png", "02406.png", "00467.png",
        "01889.png", "00466.png", "02307.png", "00529.png", "02071.png", "00045.png", "00301.png", "00491.png", "00078.png", "01289.png",
        "01862.png", "02264.png", "01297.png", "01385.png", "01348.png", "02019.png", "02099.png", "00242.png", "01101.png", "02313.png",
        "01655.png", "00923.png", "01495.png", "01947.png", "02404.png", "01293.png", "00733.png", "01579.png", "01701.png", "00248.png", 
        "01597.png", "01665.png", "01909.png", "00857.png", "00646.png", "02134.png", "00993.png", "01394.png", "01600.png", "01012.png"]

itera = 60  # Número de imagenes de prueba

list_GTA = os.listdir(path_GTA)
images_GTA = random.choices(list_GTA, k=itera)

for i in range(itera):
  images_GTA[i] = path_GTA + imgs[i]

# Automático para muchos

In [ ]:
all_iou = []
all_half_iou = []

for num in range(0, itera):

# NST  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  images_name = ["GTA/" + imgs[num], 
                 "Resultados/AA_NST_50_" + str(num+1) + ".png", "Resultados/AA_NST_75_" + str(num+1) + ".png", "Resultados/AA_NST_100_" + str(num+1) + ".png",
                 "Resultados/AA_NST_200_" + str(num+1) + ".png", "Resultados/AA_NST_500_" + str(num+1) + ".png", "Resultados/AA_NST_1000_" + str(num+1) + ".png"]

  mask_names = ((images_GTA[num].split("/")[-1] + " ") * 7).split()

  dataset = tf.data.Dataset.from_tensor_slices((images_name, mask_names))
  dataset = dataset.map(load_train, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)

  predictions = model.predict(dataset, batch_size = 10)
  predictions = np.argmax(predictions, axis = 3)

  idx = range(len(images_name))
  visualize_images = [load_image(train_images_folder_path, images_name[i], img_height, img_width) for i in idx]
  visualize_masks = [load_mask(train_mask_folder_path, mask_names[i], img_height, img_width, one_hot = False) for i in idx]
  preds = predictions[idx]

  iou_score = []
  iou_score_half = []

  for k in range(len(images_name)):
    output = one_hot_to_color_mask(preds[k], colors)

  # IoU calculation
    intersection = 0
    union = img_height * img_width

    for i in range(img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score.append(intersection/union)

  # Calcula de la mitad hacia abajo
    intersection = 0
    union = int(img_height/2) * img_width

    for i in range(int(img_height/2), img_height):
        for j in range(img_width):
          a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
          if sum(a.numpy())==3:
            intersection += 1

    iou_score_half.append(intersection/union)

  max_val_pos = iou_score_half.index(max(iou_score_half[1:]))


# GAN  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  images_name = ["Resultados/Output_GAN_6_" + str(num+1) + ".png", "Resultados/Output_GAN_10_" + str(num+1) + ".png", "Resultados/Output_GAN_12_" + str(num+1) + ".png"]

  mask_names = ((images_GTA[num].split("/")[-1] + " ") * 3).split()

  dataset = tf.data.Dataset.from_tensor_slices((images_name, mask_names))
  dataset = dataset.map(load_train, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)

  predictions = model.predict(dataset, batch_size = 10)
  predictions = np.argmax(predictions, axis = 3)

  idx = range(len(images_name))
  visualize_images = [load_image(train_images_folder_path, images_name[i], img_height, img_width) for i in idx]
  visualize_masks = [load_mask(train_mask_folder_path, mask_names[i], img_height, img_width, one_hot = False) for i in idx]
  preds = predictions[idx]

  iou_score_defin = [iou_score[0]]
  iou_score_half_defin = [iou_score_half[0]]
  iou_score_defin.append(iou_score[max_val_pos])
  iou_score_half_defin.append(iou_score_half[max_val_pos])

  for k in range(len(images_name)):
    output = one_hot_to_color_mask(preds[k], colors)
    
  # IoU calculation
    intersection = 0
    union = img_height * img_width

    for i in range(img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score_defin.append(intersection/union)
    
  # Calcula de la mitad hacia abajo
    intersection = 0
    union = int(img_height/2) * img_width

    for i in range(int(img_height/2), img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score_half_defin.append(intersection/union)

# Visualización final 1 (gráfico) &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  fig = px.Figure(data=[px.Bar(x = rango, y = iou_score_defin, name = "Completa"),
                        px.Bar(x = rango, y = iou_score_half_defin, name = "Mitad")])

  fig.update_layout(title="IoU respecto al método utilizado",
                    xaxis_title="Método",
                    yaxis_title="IoU",
                    autosize=False)
  fig.update_yaxes(range=[0, 1])
  fig.show()
  fig.write_image(path_result + "Comparacion_IoU_todos_" + str(num+1) + ".png")

  print("Completado el", str(num+1))

  all_iou.append(iou_score_defin)
  all_half_iou.append(iou_score_half_defin)

# Visualización final 2 (gráfico general) &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
iou_bar = list(np.mean(all_iou, axis=0))
iou_half_bar = list(np.mean(all_half_iou, axis=0))

fig = px.Figure(data=[px.Bar(x = rango, y = iou_bar, name = 'Completa', error_y=dict(type='percent', value=100/3)),
                      px.Bar(x = rango, y = iou_half_bar, name = 'Mitad', error_y=dict(type='percent', value=100/3))])

fig.update_layout(title="Media de todos los IoUs respecto al método utilizado",
                  xaxis_title="Método",
                  yaxis_title="IoU medio",
                  autosize=False)
fig.update_yaxes(range=[0, 1])
fig.show()
fig.write_image(path_result + "Comparacion_IoU_general.png")